<a href="https://colab.research.google.com/github/Team-Tensor/Face-Recognition/blob/main/CS419_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS419 Project : Team Tensor**
## **Comparison of Face Recognition models using Neural Networks and Principal Component Analysis**

# Imports

In [ ]:
import cv2
import numpy as np
from numpy import matlib
from matplotlib import pyplot as plt
import pandas as pd
import tensorflow as tf
import torch as torch

from google.colab import drive
from google.colab.patches import cv2_imshow
import os
from time import time
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import normalize
from sklearn.preprocessing import Normalizer, LabelEncoder

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
!pip install pytorchtools

# Loading Raw Image Data

### Extracting images to arrays and saving to **CSV**

In [ ]:
# for dirname, _, filenames in os.walk('/content/gdrive/MyDrive/yalefaces_extended'):
#     filecount=len(filenames)
#     y=torch.empty(filecount, device='cuda:0')
#     X=torch.empty(filecount, 25600, device='cuda:0')    #160x160=25600
#     i=0
#     with tqdm(total=filecount) as pbar:
#       for filename in filenames:
#             b = cv2.imread(os.path.join(dirname, filename),0)
#             b = cv2.resize(b, (160,160),interpolation = cv2.INTER_AREA)

#             X[i]=torch.from_numpy(b.ravel())
#             y[i]=int(filename[5:7])

#             pbar.update(1)
#             i=i+1
#             if i==filecount:
#               break

# np.savetxt('/content/gdrive/MyDrive/yalefaces_extended.csv', 
#            np.concatenate((X.cpu(),y.cpu()[:,None]),axis=1),
#            delimiter =", ", 
#            fmt ='%d')

### Using pre-extracted images from CSV file

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
data=np.loadtxt('/content/gdrive/MyDrive/yalefaces_extended.csv', delimiter=',')

In [ ]:
X=np.array(data[:,:-1], dtype=np.uint8)
y=data[:,-1]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y)

# Feature Extraction

## Principal Component Analysis (PCA)

In [ ]:
pca = PCA(n_components=1000).fit(x_train)
# # Plot cumulative variance captured by the principal components
# plt.plot(np.cumsum(pca.explained_variance_ratio_))  

fig, axes = plt.subplots(3, 8, figsize=(9, 4), subplot_kw={'xticks':[], 'yticks':[]})
for i, ax in enumerate(axes.flat):
    ax.imshow(pca.components_[i].reshape(160, 160), cmap='gray')
    ax.set_title("PC " + str(i+1))

Xtrain_pca = normalize(pca.transform(x_train), axis=0)
Xtest_pca = normalize(pca.transform(x_test), axis=0)

## VGG16

### Generate features using VGG16 model

In [ ]:
# n=2470
# X_feat = np.empty([n, 224, 224, 3])
# lb = np.empty(n)
# model = VGG16(weights='imagenet', include_top=False)
# for i in tqdm(range(n)):
#   j=np.random.randint(2470)
#   lb[i]=y[j]
#   img = X[j].reshape(160,160)
#   img = cv2.resize(img, (224,224),interpolation = cv2.INTER_AREA)
#   img = preprocess_input(img)

#   b_unroll = img.ravel()
#   b_color = np.matlib.repmat(b_unroll, 1, 3)
#   b_color = b_color.reshape(224, 224, 3)

#   X_feat[i]=b_color


# features=model.predict(X_feat)

# f2=np.empty([n, 25088])
# for i in tqdm(range(n)):
#   f2[i]=features[i].ravel()

# # print(f2.shape)

# np.savetxt('/content/gdrive/MyDrive/yalefaces_extended_vgg16-features2.csv', 
#            np.concatenate((f2,lb[:,None]),axis=1),
#            delimiter =", ", 
#            fmt ='%f')
# del X_feat
# del b_unroll
# del b_color
# del f2
# del lb

### Load previously generated features using VGG16

In [ ]:
data=np.loadtxt('/content/gdrive/MyDrive/yalefaces_extended_vgg16-features2.csv', delimiter=',')

In [ ]:
X_vgg=data[:,:-1]
y_vgg=data[:,-1]

In [ ]:
Xtrain_vgg, Xtest_vgg, y_train_vgg, y_test_vgg = train_test_split(X_vgg, y_vgg)

# Models

## SVM (with PCA)

In [ ]:
clf = SVC(kernel='rbf',C=1e3,gamma=1e-2)
start_time = time()
clf = clf.fit(Xtrain_pca, y_train)
print("Training Time: ", time()-start_time, "s")
y_pred_svm = clf.predict(Xtest_pca)
print(classification_report(y_test, y_pred_svm))

### Improve performance by hyperparameter tuning

In [ ]:
scorer = make_scorer(f1_score, average='macro')

tuned_parameters = [{'kernel': ['rbf','poly','linear','sigmoid'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4],
                     'C': [10, 100, 1000, 10000]}]
scores = ['precision', 'recall']
#for score in scores:

clf = GridSearchCV(SVC(), tuned_parameters, scoring=scorer)
clf.fit(Normalizer().fit(Xtrain_pca).transform(Xtrain_pca), y_train)
params=clf.best_params_
print(clf.best_params_)
cv= SVC(C=params["C"], gamma= params["gamma"], kernel=params["kernel"])
cv.fit(Normalizer().fit(Xtrain_pca).transform(Xtrain_pca), y_train)

y_pred = cv.predict(Normalizer().fit(Xtest_pca).transform(Xtest_pca))
pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose()

## SVM (with VGG16)

In [ ]:
clf = SVC(kernel='rbf',C=100,gamma=1e-2)
start_time = time()
clf = clf.fit(Xtrain_vgg, y_train_vgg)
print("Training Time: ", time()-start_time, "s")

y_pred_svm_vgg = clf.predict(Xtest_vgg)
print(classification_report(y_test_vgg, y_pred_svm_vgg))

## Feed-forward Neural Network

In [ ]:
class net(nn.Module):
  def __init__(self,input_dim):
    super(net,self).__init__()
    self.bn1=nn.BatchNorm1d(input_dim)
    self.fc1=nn.Linear(input_dim,128)
    self.g1=nn.ReLU()
    #self.bn2=nn.BatchNorm1d(128)
    #self.d1=nn.Dropout(p=0.4)
    self.fc2=nn.Linear(128,128)
    self.g2=nn.ReLU()
    #self.bn3=nn.BatchNorm1d(256)
    self.fc3=nn.Linear(128,128)
    self.g3=nn.ReLU()
    self.fc4=nn.Linear(128,39)
    #self.g4=F.softmax(out, dim=1)
    
  def forward(self, x):
    out=self.bn1(x)
    out=x
    out=self.fc1(out)
    out=self.g1(out)
    #out=F.dropout(out,p=0.2)
    #out=self.bn2(out)
    #out=self.d1(out)
    out=self.fc2(out)
    out=self.g2(out)
    #out=F.dropout(out,p=0.2)
    #out=self.bn3(out)
    out=self.fc3(out)
    out=self.g3(out)
    out=self.fc4(out)
    #out=self.g4(out)
    return out

#criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(my_net.parameters(), lr=0.003)


In [ ]:
def ffnn_output(x_train,y_train,x_test,y_test,n_iters=1000):
  x_train=x_train.astype(np.float32)
  x_test=x_test.astype(np.float32)
  input=torch.tensor(x_train,device=device)
  test_input=torch.tensor(x_test,device=device)
  labels=torch.tensor(y_train-1,dtype=torch.long,device=device)
  test_labels=torch.tensor(y_test-1,dtype=torch.long,device=device)

  loss_record=[]
  my_net=net(x_train.shape[1])
  my_net.to(device)
  m=x_train.shape[0]

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(my_net.parameters(), lr=0.003)
  start_time=time()
  for i in tqdm(range(n_iters)):
    
    optimizer.zero_grad()

    outputs = my_net(input)

    
    loss = criterion(outputs, labels)

    
    loss.backward()

    
    optimizer.step()

    loss_record.append(loss.item()/m)

    
  print("Training Time: ", time()-start_time, "s")
  plt.plot(loss_record)

  y_pred=my_net(test_input)
  y_pred=y_pred.cpu().detach().numpy()
  
  print(classification_report(test_labels.cpu(), np.argmax(y_pred,axis=1)))
  

In [ ]:
pca_X_csv=X
pca = PCA(n_components=1000).fit_transform(pca_X_csv)
pca_x_train, pca_x_test, pca_y_train, pca_y_test = train_test_split(pca, y)
ffnn_output(pca_x_train,pca_y_train, pca_x_test, pca_y_test,1000)

## Logistic Regression (PCA)

In [ ]:
x_train=normalize(Xtrain_pca, axis=0)
x_test=normalize(Xtest_pca, axis=0)

# y_train=pca_y_train 
# y_test=pca_y_test

In [ ]:
def log_reg_output(x_train,y_train,x_test,y_test):
  log_reg=LogisticRegression(max_iter=1000)

  start_time=time()
  log_reg.fit(x_train,y_train)
  print("Training Time: ", time()-start_time, "s")

  y_pred_logreg=log_reg.predict(x_test)

  print(classification_report(y_test,y_pred_logreg))

log_reg_output(x_train,y_train,x_test,y_test)

## Logistic Regression (VGG16)

In [ ]:
x_train=normalize(Xtrain_vgg, axis=0)
x_test=normalize(Xtest_vgg, axis=0)

y_train=y_train_vgg 
y_test=y_test_vgg

In [ ]:
def log_reg_output(x_train,y_train,x_test,y_test):
  log_reg=LogisticRegression(max_iter=1000)

  start_time=time()
  log_reg.fit(x_train,y_train)
  print("Training Time: ", time()-start_time, "s")

  y_pred_logreg=log_reg.predict(x_test)

  print(classification_report(y_test,y_pred_logreg))

log_reg_output(x_train,y_train,x_test,y_test)

## Convolutional Neural Network (CNN)

In [ ]:
x_train_cnn=np.resize(x_train,(x_train.shape[0],160,160))
x_test_cnn=np.resize(x_test,(x_test.shape[0],160,160))
y_train_cnn=y_train
y_test_cnn=y_test

print(x_train_cnn.shape)
print(x_test_cnn.shape)
print(y_train_cnn.shape)
print(y_test_cnn.shape)

x_train_cnn = x_train_cnn.reshape(x_train.shape[0], 1, 160, 160)
x_train_cnn=x_train_cnn.astype(np.float32)
x_train_cnn  = torch.from_numpy(x_train_cnn)

# converting the target into torch format
y_train_cnn = y_train-1
y_train_cnn = y_train_cnn.astype(int);
y_train_cnn = torch.from_numpy(y_train_cnn)

# shape of training data
x_train_cnn.shape, y_train_cnn.shape

x_test_cnn = x_test_cnn.reshape(x_test.shape[0], 1, 160, 160)
x_test_cnn=x_test_cnn.astype(np.float32)
x_test_cnn  = torch.from_numpy(x_test_cnn)

# converting the target into torch format
y_test_cnn = y_test-1
y_test_cnn = y_test_cnn.astype(int);
y_test_cnn = torch.from_numpy(y_test_cnn)

# shape of test data
x_test_cnn.shape, y_test_cnn.shape

In [ ]:
class CNN_Net(nn.Module):
  def __init__(self):
    super(CNN_Net, self).__init__()

    self.conv1=nn.Conv2d(1,12,kernel_size=5)
    self.a1=nn.ReLU()
    self.pool1=nn.MaxPool2d(2,stride=2)
    self.conv2=nn.Conv2d(12,24,kernel_size=4,stride=2)
    self.a2=nn.ReLU()
    self.pool2=nn.MaxPool2d(2)
    self.conv3=nn.Conv2d(24,32,kernel_size=3,stride=2)
    self.a3=nn.ReLU()
    self.pool3=nn.MaxPool2d(2)
    self.fc1 = nn.Linear(512, 120) 
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 39)

  def forward(self,out):
    out=self.conv1(out)
    out=self.a1(out)
    out=self.pool1(out)
    out=self.conv2(out)
    out=self.a2(out)
    out=self.pool2(out)
    out=self.conv3(out)
    out=self.a3(out)
    out=self.pool3(out)
    out = torch.flatten(out, 1)
    out=self.fc1(out)
    out=self.fc2(out)
    out=self.fc3(out)

    return out


In [ ]:
model = CNN_Net()
# defining the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

In [ ]:
from torch.autograd import Variable
x_train_new, y_train_new = Variable(x_train_cnn), Variable(y_train_cnn)
x_val_new, y_val_new = Variable(x_test_cnn[:100]), Variable(y_test_cnn[:100])

"""import pytorchtools
from pytorchtools import EarlyStopping
callbacks = [EarlyStopping(monitor='val_loss', patience=5)]
model.set_callbacks(callbacks)"""

best_loss=100000
flag=0

def train(epoch,x_train=x_train_new, y_train=y_train_new,x_val=x_val_new, y_val=y_val_new,best_loss=best_loss):
    model.train()
    tr_loss = 0
    # getting the training set
    #x_train, y_train = Variable(x_train_cnn), Variable(y_train_cnn)
    # getting the validation set
    #x_val, y_val = Variable(x_test_cnn), Variable(y_test_cnn)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train.item()/y_train.shape[0])
    val_losses.append(loss_val.item()/y_val.shape[0])

    print('Epoch : ',epoch+1, '\t', 'train_loss :', loss_train.item(),'\t', 'test_loss :', loss_val.item())

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()


    #Early stopping
    if best_loss > loss_val.item():
      best_loss= loss_val.item()
      flag = 0
      #torch.save(model.state_dict(), "model_" + str(fold) + 'weight.pt')
      torch.save(model, "/content/best_model.pt")
    else:
        flag += 1
        print("Counter {} of 5".format(es))

        if flag > 4:
          print("Early stopping with best_loss: ", best_loss, "and val_loss for this epoch: ", loss_val.item(), "...")
          return

 


    #if epoch%2 == 0:
        # printing the validation loss
        #print('Epoch : ',epoch+1, '\t', 'loss :', loss_val.item())

In [ ]:
n_epochs = 100
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.show()

In [ ]:
best_model=torch.load("/content/best_model.pt")
best_model.eval()

In [ ]:
x_test_new=Variable(x_test_cnn)
y_pred_cnn=best_model(x_test_new.cuda())
y_pred_cnn=y_pred_cnn.cpu().detach().numpy()
   
print(classification_report(y_test_cnn, np.argmax(y_pred_cnn,axis=1)))